# Анализ метрик на датасете Wikidata Big (Temporal Knowledge Graph)

Этот блокнот демонстрирует процесс адаптации существующей системы к работе с крупномасштабным темпоральным графом знаний **Wikidata Big**. 

**Основные шаги:**
1. Загрузка данных Wikidata (сущности, отношения, временные метки).
2. Отображение (Mapping) числовых ID в строковые представления Wikidata (Q-IDs, P-IDs).
3. Инициализация векторной модели и индексация темпоральных триплетов.
4. Расчет метрик качества (Hits@K, MRR) с учетом временной координаты.

In [1]:
# Конфигурация: использовать ТОЛЬКО PyTorch (не TensorFlow)
import os
os.environ['USE_TF'] = '0'
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'

print('✓ Настроен PyTorch-only режим')

✓ Настроен PyTorch-only режим


In [2]:
import sys
import os
import pickle
import numpy as np
import random
import shutil
from tqdm.auto import tqdm
import torch

# Добавление src в путь для импорта модулей проекта
sys.path.append('../../')

# Импорт необходимых компонентов системы
from src.kg_model.embeddings_model import EmbeddingsModel, EmbeddingsModelConfig, EmbedderModelConfig
from src.db_drivers.vector_driver import VectorDriverConfig, VectorDBConnectionConfig, VectorDBInstance
from src.utils.data_structs import TripletCreator, NodeCreator, NodeType, RelationCreator, RelationType
from src.utils import Logger

/Users/nmuravya/.pyenv/versions/3.11.7/lib/python3.11/site-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.27.2 is exactly one major version older than the runtime version 6.30.2 at schema.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
/Users/nmuravya/.pyenv/versions/3.11.7/lib/python3.11/site-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.27.2 is exactly one major version older than the runtime version 6.30.2 at common.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
/Users/nmuravya/.pyenv/versions/3.11.7/lib/python3.11/site-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.27.2 is exactly one major version older than the runtime version 6.30.2 at milvus.proto. Please update the gencode to avoid compatibility violations in the next runtime 

## 1. Загрузка и подготовка данных Wikidata

Данные WikidataBig хранятся в формате pickle и содержат:
- `ent_id`: словарь соответствия Q-идентификаторов (строк) числовым ID.
- `rel_id`: словарь соответствия P-идентификаторов (отношений) числовым ID.
- `ts_id`: словарь временных меток.
- `test.pickle` / `valid.pickle`: массивы триплетов с временными интервалами.

In [3]:
DATA_PATH = '../../wikidata_big/kg/tkbc_processed_data/wikidata_big/'

def load_wikidata_mapping(file_name):
    with open(os.path.join(DATA_PATH, file_name), 'rb') as f:
        return pickle.load(f)

print("Загрузка словарей соответствия...")
ent_to_id = load_wikidata_mapping('ent_id')
rel_to_id = load_wikidata_mapping('rel_id')
ts_to_id = load_wikidata_mapping('ts_id')

# Создаем обратные словари для восстановления строк по ID
id_to_ent = {v: k for k, v in ent_to_id.items()}
id_to_rel = {v: k for k, v in rel_to_id.items()}
id_to_ts = {v: str(k) for k, v in ts_to_id.items()}

print(f"Загружено сущностей: {len(ent_to_id)}")
print(f"Загружено отношений: {len(rel_to_id)}")
print(f"Загружено временных меток: {len(ts_to_id)}")

Загрузка словарей соответствия...
Загружено сущностей: 125726
Загружено отношений: 203
Загружено временных меток: 9621


In [4]:
print("Загрузка тестовых данных...")
with open(os.path.join(DATA_PATH, 'test.pickle'), 'rb') as f:
    test_data = pickle.load(f) # Формат: (s, r, o, start_t, end_t)

print(f"Количество тестовых квадруплетов: {len(test_data)}")

Загрузка тестовых данных...
Количество тестовых квадруплетов: 4995


## 2. Конвертация в формат системы (Triplet)

Мы преобразуем числовые данные Wikidata в объекты `Triplet`, используя нашу обновленную структуру с поддержкой времени.

In [5]:
def convert_to_triplets(data_subset, sample_limit=500):
    # Берем подмножество для ускорения демонстрации
    subset = data_subset[:sample_limit]
    converted = []
    
    for row in tqdm(subset, desc="Конвертация"):
        # Извлекаем названия из ID
        s_name = id_to_ent[row[0]]
        r_name = id_to_rel[row[1]]
        o_name = id_to_ent[row[2]]
        t_name = id_to_ts[row[3]] # Используем стартовое время как основной маркер
        
        # Создаем узлы и отношение
        s_node = NodeCreator.create(NodeType.object, s_name)
        r_rel = RelationCreator.create(RelationType.simple, r_name)
        o_node = NodeCreator.create(NodeType.object, o_name)
        t_node = NodeCreator.create(NodeType.time, t_name)
        
        # Создаем темпоральный триплет (S, P, O, T)
        triplet = TripletCreator.create(s_node, r_rel, o_node, time=t_node)
        converted.append(triplet)
        
    return converted

test_triplets = convert_to_triplets(test_data)
print(f"Пример: {test_triplets[0].stringified}")

Конвертация:   0%|          | 0/500 [00:00<?, ?it/s]

Пример: (1918, 0, 0): Q762215 P241 Q9212


## 3. Инициализация модели и расчет метрик

Мы инициализируем `EmbeddingsModel` и проводим поиск по векторизованному представлению запроса `(s, p, t)` для нахождения верного `o`.

In [6]:
# Конфигурация векторного хранилища (тестовая)
NODES_DB_PATH = '../../data/graph_structures/vectorized_nodes/wikidata_test'
TRIPLETS_DB_PATH = '../../data/graph_structures/vectorized_triplets/wikidata_test'
# Используем HuggingFace Hub если локальная модель отсутствует
EMBEDDER_PATH = '../../models/intfloat/multilingual-e5-small'
if not os.path.exists(EMBEDDER_PATH):
    EMBEDDER_PATH = 'intfloat/multilingual-e5-small'
# Автоопределение устройства (CUDA/MPS/CPU)
from src.utils.device_utils import get_device
DEVICE = get_device()

# Очистка предыдущих тестов
for path in [NODES_DB_PATH, TRIPLETS_DB_PATH]:
    if os.path.exists(path): shutil.rmtree(path)

config = EmbeddingsModelConfig(
    nodesdb_driver_config=VectorDriverConfig(db_config=VectorDBConnectionConfig(conn={"path": NODES_DB_PATH}, need_to_clear=True)),
    tripletsdb_driver_config=VectorDriverConfig(db_config=VectorDBConnectionConfig(conn={"path": TRIPLETS_DB_PATH}, need_to_clear=True)),
    embedder_config=EmbedderModelConfig(model_name_or_path=EMBEDDER_PATH, device=DEVICE)
)

model = EmbeddingsModel(config)
# model.embedder.init_model()

# Индексация данных
print("Индексация тестовых триплетов в векторную БД...")
model.create_triplets(test_triplets)

✓ Используется Apple Silicon GPU (MPS)


Loading weights:   0%|          | 0/199 [00:00<?, ?it/s]

BertModel LOAD REPORT from: intfloat/multilingual-e5-small
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Индексация тестовых триплетов в векторную БД...


100%|██████████| 4/4 [00:04<00:00,  1.08s/it]


{'nodes': {'2c868ab8ee77796b7331fa7ef8ba11a1',
  '1dc3143141ac99e95352d0ca32fbe2c6',
  '9779b768cd19dc47acf156ace9341748',
  '7875f1900b3965c1b9019bc0d6b9ef71',
  '6ca3237d4512f7f1cacff761699c3fe4',
  '1fcdaa5dfc9147c8990325c952b8eb1f',
  '9024a50e48b86ae3389749e88a11cb68',
  '28522b2b67d9a5a01f36312c1c468c1c',
  '43709c98cb8a025e8ed59e34619fcb78',
  'c55f50315169c5eb399ba70a9dc96b6a',
  '2ddf560b5e4c865a0c503cf1b5650eec',
  '6269eb5e1c9281ab579d968ed65644a7',
  '3477eba596514875650bbabee690138e',
  '380e5a2d88ff7ce366782c2b32b22b6f',
  'e7f68a133e9bf73162dfa375f06cedad',
  'b54198eea08efd80e153cc8ee4b58f4a',
  '1de58c94a77cbf869697a9640bd01b7e',
  '1aa10dd19e40135fefba8d1a68174f33',
  '7effc27d8d5f57aec8cc92efd2538a04',
  '99698a16e6bcf217bdf7772c07a0c59f',
  '7e248a184426d3f0f61ed9cab4bfe9b3',
  '142a0f7dcfca225d9dc6fc19077822b5',
  '73f4ea8c20c5ab96e1659580c7ae535e',
  'a808a03095d39ed146ea07642f6774a0',
  'a1462549d4cb6cd58fa04e5c892eac01',
  'da5383a1f1eaf306dd64dee4777da0fd',
  '

## 4. Расчет MRR и Hits@K

**Метрики:**
- **Hits@K**: Доля случаев, когда правильный ответ находится в топ-K результатах.
- **MRR (Mean Reciprocal Rank)**: Среднее значение величины, обратной рангу правильного ответа.

In [11]:
def evaluate_wikidata_link_prediction(model, triplets, k_values=[1, 5, 10, 100]):
    hits = {k: 0 for k in k_values}
    mrr = 0
    count = 0
    
    for triplet in tqdm(triplets, desc="Оценка"):
        # Формируем запрос: "Time: {t} | Subject Relation"
        t_str = triplet.time.name
        s_str = triplet.start_node.name
        r_str = triplet.relation.name
        query_text = f"{t_str}: {s_str} {r_str}"
        
        # Получаем эмбеддинг запроса
        q_emb = model.embedder.encode_passages([query_text])[0]
        query_inst = VectorDBInstance(id='q', document=query_text, embedding=q_emb)
        
        # Поиск ближайших узлов (кандидатов на роль Object)
        results = model.vectordbs['nodes'].retrieve([query_inst], n_results=100, includes=[])
        candidates = results[0] # Список (distance, instance)
        
        # Ищем ранг правильного объекта
        gold_id = triplet.end_node.id
        rank = None
        for i, (dist, inst) in enumerate(candidates):
            if inst.id == gold_id:
                rank = i + 1
                break
        
        if rank is not None:
            mrr += 1.0 / rank
            for k in k_values:
                if rank <= k: hits[k] += 1
        
        count += 1
    
    # Усреднение результатов
    mrr /= count
    final_hits = {k: v / count for k, v in hits.items()}
    
    return final_hits, mrr

hits, mrr = evaluate_wikidata_link_prediction(model, test_triplets)

print("\n--- РЕЗУЛЬТАТЫ WIKIDATA BIG ---")
print(f"MRR: {mrr:.5f}")
for k, val in hits.items():
    print(f"Hits@{k}: {val}")

Оценка:   0%|          | 0/500 [00:00<?, ?it/s]


--- РЕЗУЛЬТАТЫ WIKIDATA BIG ---
MRR: 0.00004
Hits@1: 0.0
Hits@5: 0.0
Hits@10: 0.0
Hits@100: 0.004
